<a target="_blank" href="https://colab.research.google.com/github/PassengerSim/algorithms/blob/main/forecasting/conditional-forecasting.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>  

This notebook defines an agreed algorithm for hybrid forecasting conditional on availability.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Exogenous Inputs to Conditional Forecast

## Timeframes

The booking horizon is divided into timeframes of arbitrary length, not necesssarily 
homogeneous. Without loss of generality, we assume each timeframe is a whole number of 
days. Each timeframe can be identified by a `DCP_INDEX`, which starts at 0 and increases
by one sequentially through the timeframes, or by a `TF` identity, which gives the
number of days from departure at the beginning of the timeframe.


In [2]:
timeframes = pd.Series([21, 14, 7], name="TF").rename_axis(index="DCP_INDEX")
timeframes.to_frame()

,TF
DCP_INDEX,
0,21
1,14
2,7


## Frat5 Curve

This curve defines, for each timeframe, the fare ratio at which half of
customers would be willing to buy up to a higher fare class.


In [3]:
frat_5_curve = pd.Series([1.20, 1.63, 2.83], index=timeframes, name="FRAT5")
frat_5_curve.to_frame()

,FRAT5
TF,
21,1.20
14,1.63
7,2.83


## Fare Classes and Prices

In [4]:
fares = pd.Series(
    {
        "Y0": 500,
        "Y1": 400,
        "Y2": 300,
        "Y3": 225,
        "Y4": 175,
        "Y5": 150,
    },
    name="PRICE",
).rename_axis(index="FARECLASS")

assert fares.is_monotonic_decreasing

fares.to_frame()

,PRICE
FARECLASS,
Y0,500
Y1,400
Y2,300
Y3,225
Y4,175
Y5,150


## Sales History

We presume we have recorded historical sales by fare class for 26 prior 
sampled days. We also have recorded whether each fare was available
for sale or not, so we can differentiate two differnt zero sales states:
because the fare class was not available, or because it was available
but we simply failed to sell it.

Conditional forecasting assumes a fully fenceless/unrestricted market, 
with the (optional) exception of advance purchase restrictions, which
apply to the time of purchase but do not otherwise differentiate fare
classes.  In this environment, there will theoretically be no sales 
above the lowest available fare class at any moment, as there is no 
reason for any customer to purchase a fare class higher than the 
lowest (least expensive) available.  In practice, sometimes "stuff happens"
and other fare classes somehow end up getting sold, but we ignore this 
for simulation. 

In this example, the Y5 class has an AP restriction at 14 days (i.e. after
the first timeframe) and the Y4 class has an AP restriction at 7 days 
(i.e. after the second timeframe).

For ease of exposition in this notebook, we comingle the sales and closure data 
here into one data structure, indicating a closure with X and a number of sales
with an integer, so that "0" means we could have sold a fare being offered but
did not, and "X" means the fare was not offered (and thus never sold).


In [5]:
X = np.nan

raw_sales_clean = dict(
    Y5=[
        [7, X, X],  # 01
        [7, X, X],  # 02
        [6, X, X],  # 03
        [X, X, X],  # 04
        [19, X, X],  # 05
        [X, X, X],  # 06
        [11, X, X],  # 07
        [X, X, X],  # 08
        [17, X, X],  # 09
        [8, X, X],  # 10
        [X, X, X],  # 11
        [23, X, X],  # 12
        [28, X, X],  # 13
        [17, X, X],  # 14
        [13, X, X],  # 15
        [X, X, X],  # 16
        [11, X, X],  # 17
        [18, X, X],  # 18
        [X, X, X],  # 19
        [X, X, X],  # 20
        [X, X, X],  # 21
        [X, X, X],  # 22
        [X, X, X],  # 23
        [X, X, X],  # 24
        [X, X, X],  # 25
        [X, X, X],  # 26
    ],
    Y4=[
        [0, 1, X],  # 01
        [0, 1, X],  # 02
        [0, 1, X],  # 03
        [3, 1, X],  # 04
        [0, 2, X],  # 05
        [5, 1, X],  # 06
        [0, 1, X],  # 07
        [6, 2, X],  # 08
        [0, 1, X],  # 09
        [0, 1, X],  # 10
        [9, X, X],  # 11
        [0, 1, X],  # 12
        [0, X, X],  # 13
        [0, 1, X],  # 14
        [0, 2, X],  # 15
        [4, X, X],  # 16
        [0, 2, X],  # 17
        [0, 1, X],  # 18
        [0, 2, X],  # 19
        [0, 1, X],  # 20
        [0, 1, X],  # 21
        [9, 2, X],  # 22
        [0, 1, X],  # 23
        [X, X, X],  # 24
        [0, 1, X],  # 25
        [X, X, X],  # 26
    ],
    Y3=[
        [0, 0, 1],  # 01
        [0, 0, 2],  # 02
        [0, 0, 0],  # 03
        [0, 0, 1],  # 04
        [0, 0, X],  # 05
        [0, 0, 1],  # 06
        [0, 0, 3],  # 07
        [0, 0, 2],  # 08
        [0, 0, 1],  # 09
        [0, 0, 2],  # 10
        [0, 1, 3],  # 11
        [0, 0, 1],  # 12
        [0, 5, X],  # 13
        [0, 0, 1],  # 14
        [0, 0, 2],  # 15
        [0, 2, 1],  # 16
        [0, 0, 1],  # 17
        [0, 0, X],  # 18
        [0, 0, X],  # 19
        [0, 0, 1],  # 20
        [0, 0, X],  # 21
        [0, 0, X],  # 22
        [0, 0, 0],  # 23
        [X, X, 1],  # 24
        [0, 0, X],  # 25
        [4, 4, 1],  # 26
    ],
    Y2=[
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 0],  # 04
        [0, 0, X],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, 0],  # 12
        [0, 0, X],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 0],  # 17
        [0, 0, X],  # 18
        [0, 0, 1],  # 19
        [0, 0, 0],  # 20
        [0, 0, 2],  # 21
        [0, 0, 1],  # 22
        [0, 0, 0],  # 23
        [1, 0, 0],  # 24
        [0, 0, 1],  # 25
        [0, 0, 0],  # 26
    ],
    Y1=[
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 0],  # 04
        [0, 0, 1],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, 0],  # 12
        [0, 0, X],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 0],  # 17
        [0, 0, 1],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 0],  # 25
        [0, 0, 0],  # 26
    ],
    Y0=[
        [0, 0, 0],  # 01
        [0, 0, 0],  # 02
        [0, 0, 0],  # 03
        [0, 0, 0],  # 04
        [0, 0, 0],  # 05
        [0, 0, 0],  # 06
        [0, 0, 0],  # 07
        [0, 0, 0],  # 08
        [0, 0, 0],  # 09
        [0, 0, 0],  # 10
        [0, 0, 0],  # 11
        [0, 0, 0],  # 12
        [0, 0, 1],  # 13
        [0, 0, 0],  # 14
        [0, 0, 0],  # 15
        [0, 0, 0],  # 16
        [0, 0, 0],  # 17
        [0, 0, 0],  # 18
        [0, 0, 0],  # 19
        [0, 0, 0],  # 20
        [0, 0, 0],  # 21
        [0, 0, 0],  # 22
        [0, 0, 0],  # 23
        [0, 0, 0],  # 24
        [0, 0, 0],  # 25
        [0, 0, 0],  # 26
    ],
)

sales_clean = pd.concat(
    {
        k: pd.DataFrame(v)
        .rename_axis(index="SAMPLE", columns="TF")
        .rename(columns={0: 21, 1: 14, 2: 7})
        for k, v in raw_sales_clean.items()
    },
    names=["FARECLASS"],
)
sales_clean

TF                  21   14   7 
FARECLASS SAMPLE                
Y5        0        7.0  NaN  NaN
          1        7.0  NaN  NaN
          2        6.0  NaN  NaN
          3        NaN  NaN  NaN
          4       19.0  NaN  NaN
...                ...  ...  ...
Y0        21       0.0  0.0  0.0
          22       0.0  0.0  0.0
          23       0.0  0.0  0.0
          24       0.0  0.0  0.0
          25       0.0  0.0  0.0

[156 rows x 3 columns]

## Closure History

We record for each fareclass/timeframe/sample, whether the fareclass was closed.
In this example, we extract the NaNs from the sales data, then tidy that dataframe
by filling in the NaNs with zeros.

In [6]:
closures = sales_clean.isna()
sales_clean = sales_clean.fillna(0).astype(int)
closures

TF                   21     14     7 
FARECLASS SAMPLE                     
Y5        0       False   True   True
          1       False   True   True
          2       False   True   True
          3        True   True   True
          4       False   True   True
...                 ...    ...    ...
Y0        21      False  False  False
          22      False  False  False
          23      False  False  False
          24      False  False  False
          25      False  False  False

[156 rows x 3 columns]

In [7]:
# check closures are ordered consistent with monotonicity rules,
# so that the closure of any fare class in any timeframe requires
# the closure of all less expensive fare classes in the same timeframe
assert (closures.loc["Y5"] >= closures.loc["Y4"]).all(axis=None)
assert (closures.loc["Y4"] >= closures.loc["Y3"]).all(axis=None)
assert (closures.loc["Y3"] >= closures.loc["Y2"]).all(axis=None)
assert (closures.loc["Y2"] >= closures.loc["Y1"]).all(axis=None)
assert (closures.loc["Y1"] >= closures.loc["Y0"]).all(axis=None)

## Lowest Available Class

For each timeframe/sample, we identify the index of the lowest fare class available.

In [8]:
top_class = fares.index[0]

lowest_available_class = pd.DataFrame(
    0, columns=closures.loc[top_class].columns, index=closures.loc[top_class].index
)
for class_number, fare_class in enumerate(fares.index[1:], start=1):
    lowest_available_class = lowest_available_class.where(
        closures.loc[fare_class], class_number
    )

lowest_available_class

TF,21,14,7
SAMPLE,,,
0,5,4,3
1,5,4,3
2,5,4,3
3,4,4,3
4,5,4,1
5,4,4,3
6,5,4,3
7,4,4,3
8,5,4,3


In [9]:
# check data is clean (no sales above lowest available class)
any_sales = sales_clean.groupby("SAMPLE").max() > 0
highest_sold_fareclass = (
    sales_clean.groupby("SAMPLE").idxmax().map(lambda x: int(x[0][1:]))
)
assert not (highest_sold_fareclass[any_sales] - lowest_available_class).any(axis=None)

# Calculated Values

In [10]:
def supc(t):
    return -np.log(0.5) / (t - 1)


supc(frat_5_curve)

TF
21    3.465736
14    1.100234
7     0.378769
Name: FRAT5, dtype: float64

In [11]:
def outer_product(s1: pd.Series, s2: pd.Series):
    """Compute the outer product of two Series as a DataFrame."""
    return pd.DataFrame(np.outer(s1, s2), index=s1.index, columns=s2.index).rename_axis(
        index=s1.index.name, columns=s2.index.name
    )

### Sellup Probability

The sellup probability starts with a customer who would definitely purchase the bottom 
fare class if it is available.  It then computes the probability that, for any fare 
class that might be the lowest currently available, what is the probability that the
customer would purchase that fare class.  So, for the bottom class, the value is 1.0,
while for progressively higher priced fares the value drops.  This sellup probability
can be computed from the Frat5 value (or other sellup computation) and the list of
all possible fare prices.

In [12]:
def sellup_prob_func(fares, frat_5_curve):
    minimum_fare = fares.min()
    df = outer_product(
        -((fares / minimum_fare) - 1),
        supc(frat_5_curve),
    )
    return np.exp(df)


sellup_prob = sellup_prob_func(fares, frat_5_curve)
sellup_prob

TF,21,14,7
FARECLASS,,,
Y0,0.000308,0.076749,0.413212
Y1,0.003100,0.159818,0.531910
Y2,0.031250,0.332793,0.684704
Y3,0.176777,0.576882,0.827468
Y4,0.561231,0.832458,0.938823
Y5,1.000000,1.000000,1.000000


### Net Sellup Probability

The net sellup probability transforms the sellup probability, expressing the
fraction of customers in any timeframe who would be expected to purchase a 
given fare class but *not* anything more expensive.  Put another way, it is
the probability of losing the sale if you close a particular fare class. 

In [13]:
def net_sellup_prob_func(fares, frat_5_curve):
    """Differences in sellup rates."""
    differences = sellup_prob.diff(axis=0)
    # use fillna, to set the values of the top fare class
    # equal to their gross values (i.e. diff from zero)
    return differences.fillna(sellup_prob)


net_sellup_prob = net_sellup_prob_func(fares, frat_5_curve)
net_sellup_prob

TF,21,14,7
FARECLASS,,,
Y0,0.000308,0.076749,0.413212
Y1,0.002793,0.083068,0.118698
Y2,0.028150,0.172976,0.152794
Y3,0.145527,0.244089,0.142765
Y4,0.384454,0.255576,0.111355
Y5,0.438769,0.167542,0.061177


### KI fare adjustment


In [14]:
def fare_adj_ki_func(sellup_prob, fares, scale_factor=1):
    """Fare adjustment using KI algorithm."""
    df = sellup_prob.mul(fares, axis=0)
    df = df.diff().div(sellup_prob.diff())
    df = df.T.fillna(fares).T
    if scale_factor != 1:
        df = df.mul(scale_factor).add(fares * (1 - scale_factor), axis=0)
    return df


fare_adj_ki = fare_adj_ki_func(sellup_prob, fares)
fare_adj_ki

TF,21,14,7
FARECLASS,,,
Y0,500.000000,500.000000,500.000000
Y1,388.986018,307.607025,51.878172
Y2,288.986018,207.607025,-48.121828
Y3,208.894707,122.744306,-134.702735
Y4,152.009402,62.140631,-196.545717
Y5,118.022407,25.783507,-233.651297


# Conditional Forecast

The "conditional" forecast is just that: a forecast that is conditioned on the lowest 
available fare class.

The first step in the conditional forecast is to use the sellup probabilities to find the 
expected (fractional) number of sales per unit demand that would have occurred in each
historical timeframe, based on the recorded lowest available fare class from each
historical timeframe.

In [15]:
def expected_sales_func(lowest_available_class):
    """Expected bookings per unit of Q demand."""
    expected_bookings = pd.DataFrame(
        0.0, columns=sales_clean.columns, index=sales_clean.index
    )
    for samp_num in range(26):
        for tf in frat_5_curve.index:
            class_num = lowest_available_class.loc[samp_num, tf]
            sup = sellup_prob.loc[f"Y{class_num}", tf]
            expected_bookings.loc[(f"Y{class_num}", samp_num), tf] = sup

    return expected_bookings


expected_sales_per_unit_demand = expected_sales_func(lowest_available_class)

Then, we can solve a linear regression for each time frame, to compute the 
mean level of "Q" demand in each timeframe.

### Option 1: Unweighted Linear Regression

This regression gives equal weight to each unit of demand, 
no matter in which fare class it is expected to show up.
This implicitly gives more weight to fitting demand at the 
lower fare classes, which have more activity when they are
open.

In [16]:
q_means = np.zeros(3)

q_mse = np.zeros(3)
q_r2 = np.zeros(3)

for i in range(3):
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand.iloc[:, i].to_frame(),
        sales_clean.iloc[:, i].to_frame(),
    )
    q_means[i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand.iloc[:, i].to_frame())
    q_mse[i] = mean_squared_error(sales_clean.iloc[:, i], y)
    q_r2[i] = r2_score(sales_clean.iloc[:, i], y)

print(" Coefficients:", q_means)
print("Mean Sq Error:", q_mse)
print("    R Squared:", q_r2)


 Coefficients: [12.48363676  1.83805027  1.6379777 ]
Mean Sq Error: [5.63516967 0.19848226 0.08390155]
    R Squared: [0.7184752  0.58852847 0.72742916]


### Option 2: LR Weighted by Inverse Sellup Probability

This regression gives equal weight to each unit of "Q" demand, 
by counting more heavily the demand that survives when forced
to buy at a higher price. This gives more weight to fitting 
demand at the higher fare classes, which have less activity 
but are more meaningful when they are active.

Without some kind of cap, this can end up giving VERY large 
weights to high value fare classes in early timeframes.

In [17]:
weights = 1 / sellup_prob.reindex(sales_clean.index.get_level_values(0))

q_means_w = np.zeros(3)
q_mse_w = np.zeros(3)
q_r2_w = np.zeros(3)

for i in range(3):
    wgt = 1 / sellup_prob.iloc[:, i].reindex(
        sales_clean.iloc[:, i].to_frame().index.get_level_values(0)
    )
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand.iloc[:, i].to_frame(),
        sales_clean.iloc[:, i].to_frame(),
        sample_weight=weights.iloc[:, i],
    )
    q_means_w[i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand.iloc[:, i].to_frame())
    q_mse_w[i] = mean_squared_error(sales_clean.iloc[:, i], y)
    q_r2_w[i] = r2_score(sales_clean.iloc[:, i], y)

print(" Coefficients:", q_means_w)
print("Mean Sq Error:", q_mse_w)
print("    R Squared:", q_r2_w)


 Coefficients: [11.6605633   1.9381824   1.65515212]
Mean Sq Error: [5.70681004 0.19951027 0.08393109]
    R Squared: [0.71489615 0.58639731 0.72733321]


We can see from the max weights that any expected or observed `Y0` activity
that emerges in the first timeframe is going to overwhelmly control the
results for that timeframe.

In [18]:
weights.max()

TF
21    3250.997354
14      13.029445
7        2.420065
dtype: float64

### Option 3: LR Weighted by Fare

Alternatively, we can balance between Options 1 and 2 by weighting
by fares.  This removes the problem of excessively large weights
from Option 2, but still tilts the result away from the unweighted
solution that implicitly overweights lower fareclasses.

In [19]:
q_means_wf = np.zeros(3)
q_mse_wf = np.zeros(3)
q_r2_wf = np.zeros(3)

for i in range(3):
    wgt = 1 / sellup_prob.iloc[:, i].reindex(
        sales_clean.iloc[:, i].to_frame().index.get_level_values(0)
    )
    m = LinearRegression(fit_intercept=False).fit(
        expected_sales_per_unit_demand.iloc[:, i].to_frame(),
        sales_clean.iloc[:, i].to_frame(),
        sample_weight=fares.reindex(sales_clean.index.get_level_values(0)),
    )
    q_means_wf[i] = m.coef_[0][0]
    y = m.predict(expected_sales_per_unit_demand.iloc[:, i].to_frame())
    q_mse_wf[i] = mean_squared_error(sales_clean.iloc[:, i], y)
    q_r2_wf[i] = r2_score(sales_clean.iloc[:, i], y)

print(" Coefficients:", q_means_wf)
print("Mean Sq Error:", q_mse_wf)
print("    R Squared:", q_r2_wf)


 Coefficients: [12.26926131  1.906933    1.66088499]
Mean Sq Error: [5.64002961 0.19896875 0.0839541 ]
    R Squared: [0.7182324  0.58751994 0.72725846]


### Possible Research Question

The above solves an independent linear regression to get forecast "Q" demand for each 
timeframe. This assumes historical sales data from other timeframes does not have
useful information.  But a multi-target regression is readily possible:

In [20]:
m = LinearRegression(fit_intercept=False).fit(
    expected_sales_per_unit_demand,
    sales_clean,
)
m.coef_

array([[13.0534334 , -2.18715886,  0.29015529],
       [-0.02898819,  1.82765287,  0.20351878],
       [-0.01363406,  0.03766003,  1.63465294]])

Our prior regression results essentially assume all the off-diagonal terms are zero.

Is this relaxation of the independent demand assumption useful?

### Q Forecast Partitioned

The mean and variance of the base "Q" demand implied by the 
coefficients of the linear regression in each timeframe are 
then partitioned to the various fare classes, according to 
the net sellup probabilties.

In [21]:
q_partitioned_mean_raw = net_sellup_prob * q_means
q_partitioned_mean_raw

TF,21,14,7
FARECLASS,,,
Y0,0.003840,0.141069,0.676832
Y1,0.034864,0.152684,0.194424
Y2,0.351409,0.317938,0.250273
Y3,1.816702,0.448648,0.233845
Y4,4.799388,0.469761,0.182397
Y5,5.477433,0.307950,0.100206


In [22]:
q_partitioned_var_raw = net_sellup_prob * q_mse
q_partitioned_var_raw

TF,21,14,7
FARECLASS,,,
Y0,0.001733,0.015233,0.034669
Y1,0.015738,0.016488,0.009959
Y2,0.158628,0.034333,0.012820
Y3,0.820068,0.048447,0.011978
Y4,2.166465,0.050727,0.009343
Y5,2.472538,0.033254,0.005133


In [23]:
# These mean and variance levels are zero'ed out whenever the adjusted fare is negative,
# and as appropriate for AP closures.
q_partitioned_mean = q_partitioned_mean_raw.where(fare_adj_ki > 0, 0)
q_partitioned_mean.loc["Y5", 14:] = 0
q_partitioned_mean.loc["Y4", 7:] = 0
q_partitioned_mean

TF,21,14,7
FARECLASS,,,
Y0,0.003840,0.141069,0.676832
Y1,0.034864,0.152684,0.194424
Y2,0.351409,0.317938,0.000000
Y3,1.816702,0.448648,0.000000
Y4,4.799388,0.469761,0.000000
Y5,5.477433,0.000000,0.000000


In [24]:
q_partitioned_var = q_partitioned_var_raw.where(fare_adj_ki > 0, 0)
q_partitioned_var.loc["Y5", 14:] = 0
q_partitioned_var.loc["Y4", 7:] = 0
q_partitioned_var

TF,21,14,7
FARECLASS,,,
Y0,0.001733,0.015233,0.034669
Y1,0.015738,0.016488,0.009959
Y2,0.158628,0.034333,0.000000
Y3,0.820068,0.048447,0.000000
Y4,2.166465,0.050727,0.000000
Y5,2.472538,0.000000,0.000000


### Total Forecast to Departure

Take the cumulative sum (backward) for the forecast mean and variance 
to get the total forecast mean and variance to departure.

In [25]:
total_forecast_mean_to_departure = (
    q_partitioned_mean.iloc[:, ::-1].cumsum(axis=1).iloc[:, ::-1]
)
total_forecast_mean_to_departure

TF,21,14,7
FARECLASS,,,
Y0,0.821741,0.817901,0.676832
Y1,0.381972,0.347108,0.194424
Y2,0.669348,0.317938,0.000000
Y3,2.265350,0.448648,0.000000
Y4,5.269149,0.469761,0.000000
Y5,5.477433,0.000000,0.000000


In [26]:
total_forecast_var_to_departure = (
    q_partitioned_var_raw.iloc[:, ::-1].cumsum(axis=1).iloc[:, ::-1]
)
total_forecast_var_to_departure

TF,21,14,7
FARECLASS,,,
Y0,0.051636,0.049903,0.034669
Y1,0.042184,0.026446,0.009959
Y2,0.205780,0.047152,0.012820
Y3,0.880493,0.060426,0.011978
Y4,2.226535,0.060070,0.009343
Y5,2.510925,0.038387,0.005133


In [27]:
total_forecast_stdev_to_departure = np.sqrt(total_forecast_var_to_departure)
total_forecast_stdev_to_departure

TF,21,14,7
FARECLASS,,,
Y0,0.227235,0.223389,0.186197
Y1,0.205388,0.162624,0.099794
Y2,0.453630,0.217146,0.113224
Y3,0.938346,0.245816,0.109445
Y4,1.492158,0.245092,0.096658
Y5,1.584590,0.195926,0.071644
